# MUHAMMAD KHUBAIB RAZA 
#### Lab 1

#### import libraries

In [0]:
import psycopg2 as psy

#### Build connection

In [0]:
conn = psy.connect(user = "postgres",
                        password = "12345678",
                        host = "127.0.0.1",
                        port = "5432")

In [0]:
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

In [0]:
#droping tabels and database
# cur.execute("""DROP Table Transactions""")
# cur.execute("""DELETE FROM Customer_Details""");


#### Creating Database

In [0]:
conn.set_session(autocommit=True)
cur.execute("""CREATE DATABASE Store""")


#### Creating Tabels

In [0]:
cur.execute("""CREATE TABLE Customer_Details ( 
    Customer_ID varchar(255), 
    Customer_name varchar(255), 
    Address varchar(255), 
    City varchar(255), 
    Country varchar(255) 

);""") 

#Transactions 

cur.execute("""CREATE TABLE Transactions ( 
    Transaction_ID varchar(255), 
    Item_ID varchar(255), 
    Customer_ID varchar(255), 
    Time date
);""") 
#Items
cur.execute("""CREATE TABLE Items (
    Item_ID varchar(255),
    Item_name varchar(255),
    Price int
    );""")

#### insert values in Table

In [0]:
#Customer table values insert
query_customer = "INSERT INTO Customer_Details (Customer_ID, Customer_name, Address, City, Country) \
                 VALUES (%s, %s, %s, %s, %s)"

data_customer = [("1","Ahmad","52-A Main Street","Lahore","Pakistan"),
        ("2","Ali","65-B Cinema Street","Multan","Pakistan"),
        ("3","Faisal","21-B Block 2","Antwerp","Belgium"),
        ("4","Mudassir","42 XX,Phase x","Lahore","Pakistan")]

try:
    cur.executemany(query_customer,data_customer)
except psy.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
#Transactions table values insert
query_transaction = "INSERT INTO Transactions (Transaction_ID, Item_ID, Customer_ID, Time) \
                 VALUES (%s, %s, %s, %s)"

data_transaction = [("1","2","1","2019-02-3"),
                    ("2","2","1","2019-02-3"),
                    ("3","3","2","2019-02-4"),
                    ("4","1","3","2019-02-3"),
                    ("5","2","4","2019-02-3")]

try:
    cur.executemany(query_transaction,data_transaction)
except psy.Error as e: 
    print("Error: Inserting Rows")
    print (e)

    
#Items table values insert
query_item = "INSERT INTO Items (Item_ID, Item_name, Price) VALUES (%s, %s, %s)"

data_item = [("1","Wall Mirror",8500),
            ("2","Master Plastic Chair",2300),
            ("3","Shoe Rack",4500)]

try:
    cur.executemany(query_item,data_item)
except psy.Error as e: 
    print("Error: Inserting Rows")
    print (e)



#### Performing Queries

In [0]:
query_a="select Customer_Details.Customer_ID,\
            Customer_Details.Customer_name,\
            sum(Price)as pp \
            from Customer_Details \
            JOIN Transactions \
            ON Customer_Details.Customer_ID=Transactions.Customer_ID \
            JOIN Items \
            ON Transactions.Item_ID=Items.Item_ID \
            group by Customer_Details.Customer_ID,Customer_Details.Customer_name \
            having sum(Price)>=3000 ;"
cur.execute(query_a)
rows=cur.fetchall()
for row in rows:
    print(row)

('1', 'Ahmad', 4600)
('2', 'Ali', 4500)
('3', 'Faisal', 8500)


In [0]:
query_b="Select * from Customer_details where City = 'Lahore'"
cur.execute(query_b)
rows=cur.fetchall()
for row in rows:
    print(row)

('1', 'Ahmad', '52-A Main Street', 'Lahore', 'Pakistan')
('4', 'Mudassir', '42 XX,Phase x', 'Lahore', 'Pakistan')


#### here i changed 5000 to 4000 becuase When I try to perform a  query, It return empty list

In [0]:
query_c="select Customer_Details.Customer_ID,\
            Customer_Details.Customer_name,\
            sum(Price)as pp \
            from Customer_Details \
            JOIN Transactions \
            ON Customer_Details.Customer_ID=Transactions.Customer_ID \
            JOIN Items \
            ON Transactions.Item_ID=Items.Item_ID \
            where City='Lahore' \
            group by Customer_Details.Customer_ID,Customer_Details.Customer_name \
            having sum(Price)>=4000 ;"
cur.execute(query_c)
rows=cur.fetchall()
for row in rows:
    print(row)

('1', 'Ahmad', 4600)


#### I changed your query Lahore with Antwerp (bcz When I try to query Lahore It returns empty list)

In [0]:

query_d="select (Customer_Details.Customer_name,Customer_Details.City,Customer_Details.Country,Items.Item_name)\
                                FROM Customer_Details \
                                JOIN Transactions \
                                ON Customer_Details.Customer_ID=Transactions.Customer_ID \
                                JOIN Items \
                                ON Transactions.Item_ID=Items.Item_ID \
                                WHERE Customer_Details.City='Antwerp' AND Items.Item_name='Wall Mirror';"
cur.execute(query_d)
rows=cur.fetchall()
for row in rows:
    print(row)

('(Faisal,Antwerp,Belgium,"Wall Mirror")',)


In [0]:
#EXtract is function of PostgreSQL
query_e="SELECT Customer_Details.Customer_name,Transactions.Transaction_ID,Transactions.Time \
                                FROM Customer_Details \
                                JOIN Transactions ON Customer_Details.Customer_ID=Transactions.Customer_ID \
                                where  EXTRACT(MONTH FROM Time)=02 AND EXTRACT(DAY FROM Time)=03;"
cur.execute(query_e)
rows=cur.fetchall()
for row in rows:
    print(row)

('Ahmad', '1', datetime.date(2019, 2, 3))
('Ahmad', '2', datetime.date(2019, 2, 3))
('Faisal', '4', datetime.date(2019, 2, 3))
('Mudassir', '5', datetime.date(2019, 2, 3))


#### Close cursor and Connection

In [0]:
cur.close()
conn.close()